# HW_2

## [요구사항1] titanic_dataset.py 분석

### titanic_dataset.py

In [107]:
import os  # 운영 체제와 상호 작용하기 위한 모듈
import pandas as pd  # 데이터 조작 및 분석을 위한 라이브러리
import torch  # PyTorch 라이브러리
from torch.utils.data import Dataset, DataLoader, random_split  # 데이터 처리 유틸리티

In [109]:
# TitanicDataset 클래스 정의: 훈련 및 검증 데이터를 위한 데이터셋 클래스
class TitanicDataset(Dataset):
    def __init__(self, X, y):
        """
        데이터셋 초기화 함수

        매개변수:
        - X: 특징 데이터 (numpy 배열 또는 pandas DataFrame)
        - y: 레이블 데이터 (numpy 배열 또는 pandas Series)
        """
        self.X = torch.FloatTensor(X)  # 특징 데이터를 부동 소수점 텐서로 변환하여 저장
        self.y = torch.LongTensor(y)   # 레이블 데이터를 정수형 텐서로 변환하여 저장

    def __len__(self):
        """데이터셋의 전체 샘플 수를 반환하는 메서드"""
        return len(self.X)

    def __getitem__(self, idx):
        """
        주어진 인덱스의 샘플을 반환하는 메서드

        매개변수:
        - idx: 가져올 샘플의 인덱스

        반환값:
        - 딕셔너리 형태로 입력과 타깃 반환
        """
        feature = self.X[idx]   # 해당 인덱스의 입력 특징
        target = self.y[idx]    # 해당 인덱스의 타깃 레이블
        return {'input': feature, 'target': target}

    def __str__(self):
        """데이터셋의 정보를 문자열로 반환하는 메서드"""
        return "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
            len(self.X), self.X.shape, self.y.shape
        )

In [111]:
# TitanicTestDataset 클래스 정의: 테스트 데이터를 위한 데이터셋 클래스
class TitanicTestDataset(Dataset):
    def __init__(self, X):
        """
        테스트 데이터셋 초기화 함수

        매개변수:
        - X: 특징 데이터 (numpy 배열 또는 pandas DataFrame)
        """
        self.X = torch.FloatTensor(X)  # 특징 데이터를 부동 소수점 텐서로 변환하여 저장

    def __len__(self):
        """데이터셋의 전체 샘플 수를 반환하는 메서드"""
        return len(self.X)

    def __getitem__(self, idx):
        """
        주어진 인덱스의 샘플을 반환하는 메서드

        매개변수:
        - idx: 가져올 샘플의 인덱스

        반환값:
        - 딕셔너리 형태로 입력 반환 (타깃 없음)
        """
        feature = self.X[idx]  # 해당 인덱스의 입력 특징
        return {'input': feature}

    def __str__(self):
        """테스트 데이터셋의 정보를 문자열로 반환하는 메서드"""
        return "Data Size: {0}, Input Shape: {1}".format(
            len(self.X), self.X.shape
        )

In [113]:
# 데이터 전처리 및 데이터셋 생성 함수 정의
def get_preprocessed_dataset():
    """
    타이타닉 데이터셋을 로드하고, 전처리하여 훈련, 검증, 테스트 데이터셋을 반환하는 함수
    """
    # 현재 파일의 디렉토리 경로 가져오기
    CURRENT_FILE_PATH = os.getcwd()
    #os.path.dirname(os.path.abspath(__file__))

    # 훈련 및 테스트 데이터 경로 설정
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    # CSV 파일에서 데이터 읽어오기
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    # 훈련 데이터와 테스트 데이터를 하나의 DataFrame으로 결합
    all_df = pd.concat([train_df, test_df], sort=False)

    # 순차적으로 전처리 함수 적용
    all_df = get_preprocessed_dataset_1(all_df)  # Fare 결측치 처리
    all_df = get_preprocessed_dataset_2(all_df)  # Name에서 호칭 추출
    all_df = get_preprocessed_dataset_3(all_df)  # Age 결측치 처리
    all_df = get_preprocessed_dataset_4(all_df)  # 가족 관련 특징 생성 및 불필요한 열 제거
    all_df = get_preprocessed_dataset_5(all_df)  # 호칭 간소화 및 Embarked 결측치 처리
    all_df = get_preprocessed_dataset_6(all_df)  # 범주형 변수 인코딩

    # 결합된 데이터를 다시 훈련 및 테스트 세트로 분할
    # Survived 열이 null이 아닌 데이터는 훈련 데이터
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]  # 원본 훈련 데이터의 Survived 열 사용

    # Survived 열이 null인 데이터는 테스트 데이터
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # 훈련 데이터셋 생성
    dataset = TitanicDataset(train_X.values, train_y.values)

    #print(dataset)
    # 훈련 데이터셋을 훈련과 검증 데이터셋으로 8:2 비율로 분할
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])

    # 테스트 데이터셋 생성
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

In [115]:
# 전처리 함수 1: Fare 결측치 처리
def get_preprocessed_dataset_1(all_df):
    """
    Pclass별 평균 Fare를 사용하여 Fare의 결측치를 채우는 함수

    매개변수:
    - all_df: 결합된 데이터프레임
    """
    # Pclass별 평균 Fare 계산
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]  # 컬럼 이름 변경

    # 평균 Fare를 원본 데이터프레임에 병합
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")

    # Fare의 결측치를 해당 Pclass의 평균 Fare로 대체
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    # 임시로 추가한 Fare_mean 열 제거
    # all_df = all_df.drop("Fare_mean", axis=1)

    return all_df

In [117]:
# 전처리 함수 2: Name에서 호칭 추출
def get_preprocessed_dataset_2(all_df):
    """
    Name 열에서 호칭(honorific)을 추출하여 새로운 열로 추가하는 함수

    매개변수:
    - all_df: 결합된 데이터프레임
    """
    # Name 열을 콤마와 점으로 분리하여 호칭 추출
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]  # 새로운 열 이름 지정

    # 각 열의 앞뒤 공백 제거
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()

    # 추출한 데이터프레임을 원본 데이터프레임에 병합
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

In [119]:
# 전처리 함수 3: Age 결측치 처리
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    """
    호칭별 중간 나이를 사용하여 Age의 결측치를 채우는 함수

    매개변수:
    - all_df: 결합된 데이터프레임
    """
    # 호칭별 중간 Age 계산
    honorific_age_median = all_df[["honorific", "Age"]].groupby("honorific").median().reset_index()
    honorific_age_median.columns = ["honorific", "honorific_age_mean"]

    # 중간 Age를 원본 DataFrame에 병합
    all_df = pd.merge(all_df, honorific_age_median, on="honorific", how="left")

    # Age의 결측치를 해당 호칭의 중간값으로 대체
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]

    # 임시로 추가한 열 삭제
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

In [121]:
# 전처리 함수 4: 가족 관련 특징 생성 및 불필요한 열 제거
def get_preprocessed_dataset_4(all_df):
    """
    가족 수와 혼자 탑승 여부를 나타내는 새로운 특징을 생성하고, 불필요한 열을 제거하는 함수

    매개변수:
    - all_df: 결합된 데이터프레임
    """
    # 가족 수 계산: 부모/자녀 수(Parch)와 형제/배우자 수(SibSp)의 합
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)

    # 모델링에 사용하지 않을 열 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

In [123]:
# 전처리 함수 5: 호칭 간소화 및 Embarked 결측치 처리
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    """
    드문 호칭을 'other'로 통합하고, Embarked 열의 결측치를 처리하는 함수

    매개변수:
    - all_df: 결합된 데이터프레임
    """
    # 흔하지 않은 호칭을 'other'로 대체
    all_df.loc[
        ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
        ),
        "honorific"
    ] = "other"

    # 'Embarked'의 결측치를 'missing'으로 채움
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df

In [125]:
# 전처리 함수 6: 범주형 변수 인코딩
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    """
    범주형 변수를 레이블 인코딩하여 수치형으로 변환하는 함수

    매개변수:
    - all_df: 결합된 데이터프레임
    """
    # 데이터프레임에서 데이터 타입이 object인 열들 선택 (범주형 변수)
    category_features = all_df.columns[all_df.dtypes == "object"]
    
    from sklearn.preprocessing import LabelEncoder # 범주형 데이터를 수치형으로 변환하는 클래스

    # 각 범주형 특징을 레이블 인코딩
    for category_feature in category_features:
        le = LabelEncoder() # 레이블 인코더 객체 생성
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature])
          all_df[category_feature] = le.transform(all_df[category_feature]) # 인코딩 후 원본 열에 저장

    return all_df

In [127]:
from torch import nn # 신경망 구축을 위한 모듈
# 신경망 모델 클래스 정의
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    """
        신경망 모델 초기화 함수

        매개변수:
        - n_input: 입력 특징의 수
        - n_output: 출력 클래스의 수 (생존 여부: 0 또는 1)
    """
    super().__init__()

    # 신경망 구조 정의
    self.model = nn.Sequential(
      nn.Linear(n_input, 30),
      nn.ReLU(),
      nn.Linear(30, 30),
      nn.ReLU(),
      nn.Linear(30, n_output),
    )

  def forward(self, x):
    """
        순전파 함수: 입력 x에 대해 모델의 출력을 계산

        매개변수:
        - x: 입력 텐서

        반환값:
        - 모델의 출력 텐서
    """
    x = self.model(x)
    return x

In [129]:
# 테스트 함수 정의
def test(test_data_loader):
    """
    테스트 데이터에 대해 모델의 예측을 수행하고 결과를 출력하는 함수

    매개변수:
    - test_data_loader: 테스트 데이터셋의 DataLoader 객체
    """
    print("[TEST]")

    # 테스트 데이터의 첫 번째 배치 가져오기
    batch = next(iter(test_data_loader))
    print("{0}".format(batch['input'].shape))  # 배치의 입력 형태 출력

    # 모델 초기화 (입력 크기는 특징 수와 일치해야 함)
    my_model = MyModel(n_input=11, n_output=2)
    ## n_input_features = batch['input'].shape[1]
    ## my_model = MyModel(n_input=n_input_features, n_output=2)  # 출력 클래스 수는 2 (생존 여부)

    # 모델에 입력을 전달하여 출력 얻기
    output_batch = my_model(batch['input'])

    # 출력 텐서에서 가장 높은 값을 가진 클래스 인덱스 추출 (예측 결과)
    prediction_batch = torch.argmax(output_batch, dim=1)

    # 예측 결과를 승객 ID와 함께 출력 (테스트 데이터의 승객 ID는 892부터 시작)
    for idx, prediction in enumerate(prediction_batch, start=892):
        print(idx, prediction.item())

In [131]:
# 메인 실행 블록: 스크립트가 직접 실행될 때만 실행
if __name__ == "__main__":
    # 전처리된 훈련, 검증, 테스트 데이터셋 가져오기
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    # 각 데이터셋의 크기 출력
    print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
        len(train_dataset), len(validation_dataset), len(test_dataset)
    ))
    print("#" * 50, 1)

    # 훈련 데이터셋의 샘플을 일부 출력하여 확인
    for idx, sample in enumerate(train_dataset):
        print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

    print("#" * 50, 2)

    # DataLoader 객체 생성: 배치 처리를 위해 데이터셋을 분할
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    print("[TRAIN]")
    # 훈련 데이터의 각 배치에 대해 입력과 타깃의 형태를 출력
    for idx, batch in enumerate(train_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("[VALIDATION]")
    # 검증 데이터의 각 배치에 대해 입력과 타깃의 형태를 출력
    for idx, batch in enumerate(validation_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("#" * 50, 3)

    # 테스트 함수 호출: 테스트 데이터에 대한 예측 수행 및 결과 출력
    test(test_data_loader)

C:\Users\User\AppData\Local\Temp\ipykernel_20796\288472932.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_20796\1577865780.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 2.0000,  1.0000,  2.0000,  1.0000,  1.0000, 26.0000,  2.0000, 21.1792,
         0.0000,  2.0000,  0.0000]): 1
1 - tensor([ 3.0000,  1.0000, 39.0000,  0.0000,  0.0000,  7.9250,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 1
2 - tensor([ 2.0000,  0.0000, 24.0000,  0.0000,  0.0000, 13.0000,  2.0000, 21.1792,
         1.0000,  0.0000,  1.0000]): 1
3 - tensor([ 3.0000,  1.0000,  9.0000,  5.0000,  2.0000, 46.9000,  2.0000, 13.3029,
         0.0000,  7.0000,  0.0000]): 0
4 - tensor([ 3.0000,  0.0000, 20.0000,  0.0000,  0.0000,  8.6625,  2.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 0
5 - tensor([ 3.0000,  1.0000, 30.0000,  0.0000,  0.0000,  8.0500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
6 - tensor([ 1.0000,  1.0000, 71.0000,  0.0000,  0.0000, 49.5042,  0.0000, 87.5090,
         2.0000,  0.0000,  1.0000]): 0
7 - tensor([ 2.00

## [요구사항2] titanic 딥러닝 모델 훈련 코드 및 Activation Function 변경해보기

In [163]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse
import os
import pandas as pd
from pathlib import Path

# BASE_PATH 설정 (필요에 따라 조정)
BASE_PATH = str(Path(__file__).resolve().parent.parent.parent)
print(BASE_PATH, "!!!!!!!")
import sys
sys.path.append(BASE_PATH)

# 전처리된 데이터셋을 가져오는 함수 정의
def get_preprocessed_dataset():
    CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))

    # 데이터 파일 경로 설정
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    # CSV 파일 로드
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    # 학습 데이터와 테스트 데이터를 하나의 데이터프레임으로 결합
    all_df = pd.concat([train_df, test_df], sort=False)

    # 각 전처리 단계 수행
    all_df = get_preprocessed_dataset_1(all_df)
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)

    # 학습용 데이터와 타겟 분리
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    # 테스트용 데이터 분리
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # 데이터셋 객체 생성
    dataset = TitanicDataset(train_X.values, train_y.values)

    # 학습 데이터셋과 검증 데이터셋으로 분할 (80:20 비율)
    train_size = int(len(dataset) * 0.8)
    validation_size = len(dataset) - train_size
    train_dataset, validation_dataset = random_split(dataset, [train_size, validation_size])

    # 테스트 데이터셋 객체 생성
    test_dataset = TitanicTestDataset(test_X.values)

    return train_dataset, validation_dataset, test_dataset

# 결측된 Fare 값을 각 Pclass의 평균값으로 채우는 함수
def get_preprocessed_dataset_1(all_df):
    # Pclass별 평균 Fare 계산
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    # 원본 데이터프레임과 병합
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    # 결측된 Fare 값을 평균값으로 채움
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]
    return all_df

# Name 열을 분리하여 새로운 열로 추가하는 함수
def get_preprocessed_dataset_2(all_df):
    # Name 열을 콤마와 점을 기준으로 분리
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    # 문자열 공백 제거
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    # 원본 데이터프레임과 병합
    all_df = pd.concat([all_df, name_df], axis=1)
    return all_df

# 결측된 Age 값을 honorific(칭호) 별 중앙값으로 채우는 함수
def get_preprocessed_dataset_3(all_df):
    # honorific 별 Age 중앙값 계산
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean"]
    # 원본 데이터프레임과 병합
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    # 결측된 Age 값을 중앙값으로 채움
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    # 임시로 사용한 열 삭제
    all_df = all_df.drop(["honorific_age_mean"], axis=1)
    return all_df

# 가족 수와 혼자 탑승 여부를 나타내는 새로운 열 생성
def get_preprocessed_dataset_4(all_df):
    # 가족 수 계산
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]
    # 혼자 탑승 여부 표시 (family_num이 0이면 혼자)
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    # 나머지는 혼자가 아님 (0으로 채움)
    all_df["alone"].fillna(0, inplace=True)
    # 불필요한 열 삭제
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)
    return all_df

# honorific의 종류를 줄이고, 결측된 Embarked 값을 처리하는 함수
def get_preprocessed_dataset_5(all_df):
    # 주요 honorific만 남기고 나머지는 'other'로 분류
    all_df.loc[
        ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
        ),
        "honorific"
    ] = "other"
    # 결측된 Embarked 값을 'missing'으로 채움
    all_df["Embarked"].fillna("missing", inplace=True)
    return all_df

# 범주형 변수를 수치형으로 변환하는 함수
def get_preprocessed_dataset_6(all_df):
    # 데이터프레임에서 object 타입의 열 선택
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
            le = le.fit(all_df[category_feature])
            all_df[category_feature] = le.transform(all_df[category_feature])
    return all_df

# 학습용 데이터셋 클래스 정의
class TitanicDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        # 입력 데이터와 타겟을 텐서로 변환
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)

    def __len__(self):
        # 데이터셋의 길이 반환
        return len(self.X)

    def __getitem__(self, idx):
        # 인덱스에 해당하는 데이터 반환
        feature = self.X[idx]
        target = self.y[idx]
        return {'input': feature, 'target': target}

    def __str__(self):
        # 데이터셋 정보 문자열로 반환
        return "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
            len(self.X), self.X.shape, self.y.shape
        )

# 테스트용 데이터셋 클래스 정의
class TitanicTestDataset(torch.utils.data.Dataset):
    def __init__(self, X):
        # 입력 데이터 텐서로 변환
        self.X = torch.FloatTensor(X)

    def __len__(self):
        # 데이터셋의 길이 반환
        return len(self.X)

    def __getitem__(self, idx):
        # 인덱스에 해당하는 데이터 반환
        feature = self.X[idx]
        return {'input': feature}

    def __str__(self):
        # 데이터셋 정보 문자열로 반환
        return "Data Size: {0}, Input Shape: {1}".format(
            len(self.X), self.X.shape
        )

# 데이터 로더를 얻는 함수 정의
def get_data():
    # 전처리된 데이터셋 가져오기
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    # 데이터셋 정보 출력
    print("Train Dataset: ", train_dataset)
    print("Validation Dataset: ", validation_dataset)
    print("Test Dataset: ", test_dataset)
    print(len(train_dataset), len(validation_dataset), len(test_dataset))

    # 학습 데이터 로더 생성
    train_data_loader = DataLoader(
        dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True
    )
    # 검증 데이터 로더 생성
    validation_data_loader = DataLoader(
        dataset=validation_dataset, batch_size=len(validation_dataset)
    )

    return train_data_loader, validation_data_loader

# 모델 클래스 정의
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        # 신경망 구조 정의
        self.model = nn.Sequential(
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            #ReLU, ELU, LeakyRelU, PReLU 변경
            nn.ReLU(), 
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.ReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
        )

    def forward(self, x):
        # 순전파 정의
        x = self.model(x)
        return x

# 모델과 옵티마이저를 얻는 함수 정의
def get_model_and_optimizer():
    # 입력 크기와 출력 크기를 지정하여 모델 생성
    my_model = MyModel(n_input=11, n_output=2)
    # SGD 옵티마이저 사용
    optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)
    return my_model, optimizer

# 훈련 루프 정의
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.CrossEntropyLoss()  # 분류 문제이므로 CrossEntropyLoss 사용
    next_print_epoch = 100

    for epoch in range(1, n_epochs + 1):
        # 훈련 손실 초기화
        loss_train = 0.0
        num_trains = 0
        for train_batch in train_data_loader:
            # 입력과 타겟 추출
            inputs = train_batch['input']
            targets = train_batch['target']
            # 모델에 입력을 통과시켜 출력 얻기
            output_train = model(inputs)
            # 손실 계산
            loss = loss_fn(output_train, targets)
            # 손실 합산
            loss_train += loss.item()
            num_trains += 1

            # 역전파 및 가중치 업데이트
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # 검증 손실 초기화
        loss_validation = 0.0
        num_validations = 0
        with torch.no_grad():
            for validation_batch in validation_data_loader:
                inputs = validation_batch['input']
                targets = validation_batch['target']
                output_validation = model(inputs)
                loss = loss_fn(output_validation, targets)
                loss_validation += loss.item()
                num_validations += 1

        # WandB에 손실 값 로깅
        wandb.log({
            "Epoch": epoch,
            "Training loss": loss_train / num_trains,
            "Validation loss": loss_validation / num_validations
        })

        # 지정된 에폭마다 출력
        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training loss {loss_train / num_trains:.4f}, "
                f"Validation loss {loss_validation / num_validations:.4f}"
            )
            next_print_epoch += 100

# 메인 함수 정의
def main(args):
    # 현재 시간 문자열로 저장
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    # 하이퍼파라미터 설정
    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [20, 20],
    }

    # WandB 초기화
    wandb.init(
        mode="online",  # 항상 온라인 모드로 설정
        project="my_model_training",
        notes="My first wandb experiment",
        tags=["my_model", "titanic_dataset"],
        name=current_time_str,
        config=config
    )
    print(args)
    print(wandb.config)

    # 데이터 로더 얻기
    train_data_loader, validation_data_loader = get_data()
    # 모델과 옵티마이저 얻기
    linear_model, optimizer = get_model_and_optimizer()

    print("#" * 50, 1)

    # 훈련 시작
    training_loop(
        model=linear_model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader
    )
    # WandB 종료
    wandb.finish()

# 스크립트 시작점
if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    # wandb 사용 여부 옵션 (현재는 항상 온라인 모드이므로 주석 처리)
    # parser.add_argument(
    #     "--wandb", action=argparse.BooleanOptionalAction, default=False, help="True or False"
    # )

    # 배치 크기 인자
    parser.add_argument(
        "-b", "--batch_size", type=int, default=512, help="Batch size (int, default: 512)"
    )

    # 에폭 수 인자
    parser.add_argument(
        "-e", "--epochs", type=int, default=1000, help="Number of training epochs (int, default:1000)"
    )

    args = parser.parse_args()
    main(args)


NameError: name '__file__' is not defined

Training loss, Validation loss 그래프를 보여주는 URL:
https://wandb.ai/rkddkwlwhd-korea-university-of-technology-and-education/my_model_training?nw=nwuserrkddkwlwhd

그래프를 통해 더 나은 성능을 산출하는 Activation Function 조사
-> ReLU 선택

## [요구사항3] 테스트 및 submission.csv 생성

In [161]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse
import os
import pandas as pd
from pathlib import Path

# BASE_PATH 설정 (필요에 따라 조정)
BASE_PATH = str(Path(__file__).resolve().parent.parent.parent)
print(BASE_PATH, "!!!!!!!")
import sys
sys.path.append(BASE_PATH)

# 데이터 전처리 및 데이터셋 생성 함수 정의
def get_preprocessed_dataset():
    # 현재 파일의 경로 가져오기
    CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))

    # train.csv와 test.csv의 경로 설정
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    # CSV 파일을 데이터프레임으로 로드
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    # train과 test 데이터를 병합하여 일관된 전처리 적용
    all_df = pd.concat([train_df, test_df], sort=False)

    # 각 전처리 단계 함수 호출
    all_df = get_preprocessed_dataset_1(all_df)
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)

    # Survived 열을 기준으로 train과 test 데이터 분리
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # 데이터셋 생성
    dataset = TitanicDataset(train_X.values, train_y.values)

    # 훈련 데이터와 검증 데이터로 분할 (80:20 비율)
    train_size = int(len(dataset) * 0.8)
    validation_size = len(dataset) - train_size
    train_dataset, validation_dataset = random_split(dataset, [train_size, validation_size])

    # 테스트 데이터셋 생성
    test_dataset = TitanicTestDataset(test_X.values)

    return train_dataset, validation_dataset, test_dataset

def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare의 평균으로 결측치 채우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]
    return all_df

def get_preprocessed_dataset_2(all_df):
    # Name 열을 분리하여 성씨, 호칭, 이름 열 생성
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)
    return all_df

def get_preprocessed_dataset_3(all_df):
    # 호칭별 Age의 중앙값으로 결측치 채우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean"]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)
    return all_df

def get_preprocessed_dataset_4(all_df):
    # 가족 수와 혼자인지 여부를 나타내는 열 생성
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)
    # 필요 없는 열 삭제
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)
    return all_df

def get_preprocessed_dataset_5(all_df):
    # 호칭의 종류를 주요 카테고리로 축소
    all_df.loc[
        ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
        ),
        "honorific"
    ] = "other"
    # Embarked의 결측치 채우기
    all_df["Embarked"].fillna("missing", inplace=True)
    return all_df

def get_preprocessed_dataset_6(all_df):
    # 범주형 변수를 LabelEncoder를 사용하여 수치형으로 변환
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
            le = le.fit(all_df[category_feature])
            all_df[category_feature] = le.transform(all_df[category_feature])
    return all_df

# 학습용 데이터셋 클래스 정의
class TitanicDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        # 입력 데이터와 타겟 데이터를 Tensor로 변환
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)

    def __len__(self):
        # 데이터셋의 크기 반환
        return len(self.X)

    def __getitem__(self, idx):
        # 인덱스에 해당하는 데이터 반환
        feature = self.X[idx]
        target = self.y[idx]
        return {'input': feature, 'target': target}

    def __str__(self):
        # 데이터셋 정보 문자열로 반환
        return "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
            len(self.X), self.X.shape, self.y.shape
        )

# 테스트용 데이터셋 클래스 정의
class TitanicTestDataset(torch.utils.data.Dataset):
    def __init__(self, X):
        # 입력 데이터만 Tensor로 변환
        self.X = torch.FloatTensor(X)

    def __len__(self):
        # 데이터셋의 크기 반환
        return len(self.X)

    def __getitem__(self, idx):
        # 인덱스에 해당하는 데이터 반환
        feature = self.X[idx]
        return {'input': feature}

    def __str__(self):
        # 데이터셋 정보 문자열로 반환
        return "Data Size: {0}, Input Shape: {1}".format(
            len(self.X), self.X.shape
        )

# 데이터로더 생성 함수 정의
def get_data():
    # 훈련 및 검증 데이터셋 로드
    train_dataset, validation_dataset, _ = get_preprocessed_dataset()

    print("Train Dataset: ", train_dataset)
    print("Validation Dataset: ", validation_dataset)
    print(len(train_dataset), len(validation_dataset))

    # 데이터로더 생성
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))

    return train_data_loader, validation_data_loader

def get_test_data_loader():
    # 테스트 데이터셋 로드
    _, _, test_dataset = get_preprocessed_dataset()
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=wandb.config.batch_size)
    return test_data_loader

# 모델 클래스 정의
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        # 모델 구성: 입력층, 은닉층, 출력층
        self.model = nn.Sequential(
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            nn.ReLU(),  # 활성화 함수로 ReLU 사용
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.ReLU(),  # 활성화 함수로 ReLU 사용
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
        )

    def forward(self, x):
        # 순전파 계산
        x = self.model(x)
        return x

# 모델과 옵티마이저를 생성하는 함수 정의
def get_model_and_optimizer():
    # 모델 생성 (입력 크기 11, 출력 크기 2)
    my_model = MyModel(n_input=11, n_output=2)
    # 옵티마이저로 SGD 사용
    optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)
    return my_model, optimizer

# 훈련 루프 정의
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.CrossEntropyLoss()  # 분류 문제이므로 CrossEntropyLoss 사용
    next_print_epoch = 100
    best_val_loss = float('inf')  # 최저 검증 손실 초기화
    best_epoch = 0  # 최적의 에포크 초기화

    for epoch in range(1, n_epochs + 1):
        model.train()  # 모델을 학습 모드로 설정
        loss_train = 0.0
        num_trains = 0
        for train_batch in train_data_loader:
            inputs = train_batch['input']
            targets = train_batch['target']
            output_train = model(inputs)
            loss = loss_fn(output_train, targets)
            loss_train += loss.item()
            num_trains += 1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()  # 모델을 평가 모드로 설정
        loss_validation = 0.0
        num_validations = 0
        with torch.no_grad():
            for validation_batch in validation_data_loader:
                inputs = validation_batch['input']
                targets = validation_batch['target']
                output_validation = model(inputs)
                loss = loss_fn(output_validation, targets)
                loss_validation += loss.item()
                num_validations += 1

        avg_train_loss = loss_train / num_trains
        avg_val_loss = loss_validation / num_validations

        # WandB에 손실 값 로깅
        wandb.log({
            "Epoch": epoch,
            "Training loss": avg_train_loss,
            "Validation loss": avg_val_loss
        })

        # 검증 손실이 개선되면 모델 저장
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_epoch = epoch
            torch.save(model.state_dict(), 'best_model.pth')  # 모델 가중치 저장
            print(f"Epoch {epoch}: Validation loss improved to {best_val_loss:.4f}. Model saved.")

        # 지정된 에폭마다 진행 상황 출력
        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training loss {avg_train_loss:.4f}, "
                f"Validation loss {avg_val_loss:.4f}"
            )
            next_print_epoch += 100

    print(f"Training complete. Best validation loss: {best_val_loss:.4f} at epoch {best_epoch}.")

# 테스트 데이터로 예측하고 submission.csv 생성하는 함수 정의
def generate_submission(model, test_data_loader):
    # 테스트 데이터에 대한 예측 수행
    predictions = []
    with torch.no_grad():
        for test_batch in test_data_loader:
            inputs = test_batch['input']
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)  # 각 샘플에 대해 확률이 높은 클래스 선택
            predictions.extend(predicted.numpy())

    # PassengerId를 가져오기 위해 원본 test.csv 로드
    CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")
    test_df = pd.read_csv(test_data_path)
    submission = pd.DataFrame({
        'PassengerId': test_df['PassengerId'],
        'Survived': predictions
    })

    # submission.csv 파일로 저장
    submission.to_csv('submission.csv', index=False)
    print("submission.csv 파일이 생성되었습니다.")

# 메인 함수 정의
def main(args):
    # 현재 시간 문자열 생성
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    # 하이퍼파라미터 설정
    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [20, 20],
    }

    # wandb 초기화
    wandb.init(
        mode="online",
        project="my_model_training",
        notes="My first wandb experiment",
        tags=["my_model", "titanic_dataset"],
        name=current_time_str,
        config=config
    )
    print(args)
    print(wandb.config)

    # 데이터로더 생성
    train_data_loader, validation_data_loader = get_data()
    linear_model, optimizer = get_model_and_optimizer()

    print("#" * 50, 1)

    # 모델 훈련
    training_loop(
        model=linear_model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader
    )

    # 저장된 최적의 모델 로드
    linear_model.load_state_dict(torch.load('best_model.pth'))
    linear_model.eval()

    # 테스트 데이터로더 생성
    test_data_loader = get_test_data_loader()

    # submission.csv 생성
    generate_submission(model=linear_model, test_data_loader=test_data_loader)

    wandb.finish()

if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "-b", "--batch_size", type=int, default=512, help="Batch size (int, default: 512)"
    )

    parser.add_argument(
        "-e", "--epochs", type=int, default=1000, help="Number of training epochs (int, default:1000)"
    )

    args = parser.parse_args()
    main(args)


NameError: name '__file__' is not defined

>기술적 사항 및 고찰 내용:

최적의 에포크 선택

고찰: 모델 훈련 시 모든 에포크에서의 모델을 사용하여 테스트를 수행할 수 있지만, 일반적으로 검증 손실이 가장 낮은 에포크에서의 모델이 테스트 데이터에 대해 가장 좋은 성능을 보입니다. 따라서 훈련 과정에서 검증 손실이 개선될 때마다 모델을 저장하고, 훈련이 완료된 후 저장된 최적의 모델을 로드하여 테스트를 수행하는 것이 바람직합니다.

추가 코딩: 위 코드에서는 best_val_loss와 best_epoch 변수를 사용하여 최적의 모델을 저장하고 로드하는 로직을 추가하였습니다.

## [요구사항4] submission.csv 제출 및 등수확인

![kaggle_titanic_jonghyeonkim](kaggle_titanic_jonghyeonkim.png)

## [요구사항5] Wandb 페이지 생성 PDF 인쇄

추가 파일 제출

## 숙제 후기

우선, Titanic 데이터셋의 특성상 결측치와 범주형 변수가 많아, 데이터 전처리가 모델의 성능에 큰 영향을 미쳤습니다. Age나 Fare와 같은 결측된 값을 채우고, Embarked와 같은 범주형 변수는 수치형으로 변환하여 모델이 학습하기 좋은 데이터로 정제하는 과정이 필요했습니다. 이와 같은 전처리를 통해 모델의 학습 효율이 향상될 수 있었고, 데이터 일관성을 유지하는 것도 중요하다는 것을 배웠습니다.모델 구조와 하이퍼파라미터 설정에서는 이진 분류 문제에 맞추어 출력 노드를 2개로 설정하고, 활성화 함수로는 ReLU를 사용했습니다. 또한, 은닉층의 노드 수나 학습률과 같은 하이퍼파라미터를 설정하고, 최적의 값을 찾기 위해 WandB를 활용하여 다양한 조합을 실험했습니다. 이 과정을 통해 하이퍼파라미터 튜닝이 모델 성능에 큰 차이를 줄 수 있음을 경험했습니다. 과적합을 방지하고 최적의 모델을 선택하기 위해서는 검증 손실을 지속적으로 모니터링하며, 최저 손실을 기록한 모델을 저장하는 로직을 추가했습니다. 이를 통해 훈련 데이터에 과적합되지 않은 모델을 최적의 에포크에서 저장하여 테스트 데이터에 적용할 수 있었습니다. 또한, 테스트 데이터에 대해 예측을 수행하고 결과를 submission.csv로 저장하는 과정에서는, 모델을 평가 모드로 전환하고 각 샘플에 대해 높은 확률을 가진 클래스를 선택하는 방식으로 구현했습니다. 이를 통해 모델의 예측 결과를 제출 형식에 맞춰 손쉽게 저장할 수 있었으며, Kaggle 등의 플랫폼에서 손쉽게 평가할 수 있었습니다.